# Exploring `base_network.py` outputs

This jupyter notebook analyse the `base_network.py` **outputs**.

The `pypsa-africa/Snakefile` explicitly list in the **rule** what goes into the function `base_network.py` and what goes out (`networks/base.nc`). When inputs are commented out (#) than they are currently not considered. Maybe you want to delete them, make them run or develop new features? What can be observed from the inputs is that the script creates a connected network topology with the network data and shapes of the region.

```
rule base_network:
    input:
        osm_buses="data/base_network/africa_all_buses_build_network.csv",
        osm_lines="data/base_network/africa_all_lines_build_network.csv",
        country_shapes='resources/country_shapes.geojson',
        offshore_shapes='resources/offshore_shapes.geojson',
        # eg = ENTSO-E Gridkit
        # eg_buses='data/entsoegridkit/buses.csv',
        # eg_lines='data/entsoegridkit/lines.csv',
        # eg_links='data/entsoegridkit/links.csv',
        # eg_converters='data/entsoegridkit/converters.csv',
        # eg_transformers='data/entsoegridkit/transformers.csv',
        # parameter_corrections='data/parameter_corrections.yaml',
        # links_p_nom='data/links_p_nom.csv',
        # links_tyndp='data/links_tyndp.csv',
        # africa_shape='resources/africa_shape.geojson'
    output:
        "networks/base.nc",
    log:
        "logs/base_network.log",
    benchmark:
        "benchmarks/base_network"
    threads: 1
    resources:
        mem=500,
    script:
        "scripts/base_network.py"
```

Before analysing the outputs of add_electricity.py check that:
- `pypsa-africa` environment (/kernel) in jupyter notebook  is active and updated
- root folder where pypsa-africa is installed is named "pypsa-africa"
- or rename the below `sets_path_to_root("<folder_name>")` accordingly

In [1]:
import sys

sys.path.append("../")  # to import helpers
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")

This is the repository path:  /home/max/OneDrive/PHD-Flexibility/07_pypsa-africa/0github/pypsa-africa/pypsa-africa
Had to go 1 folder(s) up.


A jupyter notebook requires the user to import all they need. So we need to import all the required dependencies from the `pypsa-africa` environment:

In [ ]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)

In [ ]:
import pypsa
import geopandas as gpd
import sys

sys.path.append("../")  # to import helpers
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")

## Network analysis

Now let's import the output file which should be in i.e. `~/pypsa-africa/networks/base.nc`. 

Note: If you don't have the /elec.nc file yet in your folder structure, you could try to generate it from the terminal. In this case set the path so that you are located at `~/pypsa-africa`. Once you are there, use the command:
```
snakemake -j 1 networks/base.nc
```
Note that this is something you can to for any other rule to generate it's outputs `snakemake -j 1 <output of rule>`

In [ ]:
network_path = os.getcwd() + "/networks/base.nc"
n = pypsa.Network(network_path)
n

You might be interested to see what's in the Network Common Data Form or NetCDF (.nc) file. To assess that we use **xarray**

In [ ]:
import xarray as xr

path = os.getcwd() + "/networks/base.nc"
network_data = xr.open_dataset(path)
network_data

In [ ]:
n.buses

Example to check the input:

In [ ]:
regions = (
    os.getcwd() + "/resources/country_shapes.geojson"
)  # path of the input as shown in the Snakefile
gpd.read_file(regions).plot()

That's how the raw data looks like:

In [ ]:
regions = os.getcwd() + "/resources/country_shapes.geojson"
gpd.read_file(regions)

Checking which countries are considered:

In [ ]:
n.buses["country"].unique()

Looking at the lines you find out that some of the names at the end of the pandadataframe are quite long. These are articifial created lines that create a connected network topology:

In [ ]:
n.lines

# Check hv.plot (need to be fixed)

In [ ]:
# Issue probably with ["geometry"]
n.lines["geometry"]  # .apply(wkt.loads)

In [ ]:
from shapely import wkt

lines = n.lines
lines["geometry"] = lines["geometry"].apply(wkt.loads)
lines = gpd.GeoDataFrame(lines, crs="epsg:4326")

buses = n.buses
buses["geometry"] = gpd.points_from_xy(buses.lon, buses.lat)
buses = gpd.GeoDataFrame(buses, crs="epsg:4326")

In [ ]:
lines = lines[lines["country"] == "nigeria"]
buses = buses[buses["country"] == "nigeria"]

buses.hvplot(
    geo=True,
    size=10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles="CartoLight",
    hover_cols=["bus_id"],
    color="orange",
) * lines.hvplot(geo=True, alpha=0.4, hover_cols=["line_id"]).opts(
    active_tools=["pan", "wheel_zoom"]
)